# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.7

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import re 
from tqdm import tqdm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
import spacy

In [2]:
data = pd.read_csv('toxic_comments.csv')

In [3]:
data.sample(10)

,Unnamed: 0,text,toxic
9111,9124,"""\nSorry not to see you at the session; the lo...",0
79679,79755,"""\nNPOV editing\nI'm sorry i was about to put ...",0
125367,125496,"""\n\n Jaitapur Nuclear Power Project \nHi Yoni...",0
32045,32085,If you're allowed to remove prods... \n\n...wh...,0
85034,85115,Please do not remove discussion posts \n\nThis...,0
117196,117295,"""Leslie Stefanson==\nYou'll notice the link to...",0
44805,44858,"""\n\nWith all due respect, your comments about...",0
142511,142664,I fail to understand why people immediatly jum...,0
80042,80118,"""\n\nWhy don't we try to resolve this issue on...",0
116345,116444,Accept my apology! \n\nAccept my apology THIS ...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.duplicated().sum()

0

Посмотрим на баланс классов.

In [6]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Классы явно несбалансированы. Это необходимо учесть при обучении моделей.

В нашем распоряжении датасет, состоящий из 2 столбцов и 159571 строки. В столбце text находится текст комментария, в столбце toxic — целевой признак. Дубликатов нет.

**Проведем предобработку текстовых данных**

Создадим корпус постов:

In [7]:
corpus = data['text']

Проведем лемматизацию текста и очистим текст лишних символов с помощью регулярных выражений:

In [8]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def clear_text(text):
    doc = nlp(text)
    lemm_text = ' '.join([token.lemma_ for token in doc])
    clear_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return ' '.join(clear_text.split())

In [9]:
for i in tqdm(range(len(corpus))):
    corpus[i] = clear_text(corpus[i])

100%|██████████████████████████████████| 159292/159292 [16:51<00:00, 157.40it/s]


In [10]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,explanation why the edit make under my usernam...,0
1,1,d aww he match this background colour I be see...,0
2,2,hey man I be really not try to edit war it be ...,0
3,3,More I can not make any real suggestion on imp...,0
4,4,you sir be my hero any chance you remember wha...,0
5,5,congratulation from I as well use the tool wel...,0
6,6,COCKSUCKER before you pis around on my work,1
7,7,your vandalism to the Matt Shirvington article...,0
8,8,sorry if the word nonsense be offensive to you...,0
9,9,alignment on this subject and which be contrar...,0


Разделим данные на выборки:

In [11]:
features = data['text']
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.20, random_state=111, stratify=target)

In [12]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(127433,)
(31859,)
(127433,)
(31859,)


Рассчитаем TF-IDF, указав стоп-слова:

In [13]:
nltk.download('stopwords') 
stopwords = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anastasia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, max_features=50000) 

In [15]:
tf_idf_train = count_tf_idf.fit_transform(features_train) 
tf_idf_test = count_tf_idf.transform(features_test) 

In [16]:
print(tf_idf_train.shape)
print(tf_idf_test.shape)

(127433, 50000)
(31859, 50000)


Данные разделены на выборки, признаки получены из TF-IDF. Можно приступать к обучению моделей.

## Обучение

Обучим 2 модели: LogisticRegression и RandomForestClassifier

**1) LogisticRegression**

In [25]:
%%time

lr = LogisticRegression(class_weight='balanced', random_state=222)
parameters = {
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
    'C': np.arange(0.1, 1, 0.2)
}
CV_lr = GridSearchCV(estimator=lr, param_grid=parameters, n_jobs=-1, verbose=3, cv=3, scoring='f1')
CV_lr.fit(tf_idf_train, target_train)
CV_lr_best_par = CV_lr.best_params_
CV_lr_best_score = CV_lr.best_score_
print(CV_lr_best_par, 'F1 LogisticRegression', CV_lr_best_score)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


/Users/anastasia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anastasia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

{'C': 0.9000000000000001, 'solver': 'lbfgs'} F1 LogisticRegression 0.7456337493935784
CPU times: user 11.6 s, sys: 3.73 s, total: 15.3 s
Wall time: 18 s


Проверим качество модели на тестовой выборке:

In [26]:
%%time

lr = LogisticRegression(class_weight='balanced', random_state=222, solver='lbfgs', C=0.9)
lr.fit(tf_idf_train, target_train)

CPU times: user 11.3 s, sys: 3.03 s, total: 14.3 s
Wall time: 2.1 s


LogisticRegression(C=0.9, class_weight='balanced', random_state=222)

In [27]:
%%time

lr_predictions = lr.predict(tf_idf_test)
print('F1 LogisticRegression на тестовой выборке:', f1_score(target_test, lr_predictions))

F1 LogisticRegression на тестовой выборке: 0.7340354619384083
CPU times: user 118 ms, sys: 71 ms, total: 189 ms
Wall time: 40.1 ms


На тестовой выборке модель LogisticRegression показала метрику качества F1 = 0.73.

**2) RandomForestClassifier**

In [28]:
%%time

rf = RandomForestClassifier(class_weight='balanced', random_state=12345)
parameters = {
    'n_estimators': list(range(50, 200, 50)),
    'max_depth': list(range(1, 20, 5))
}
CV_rf = GridSearchCV(estimator=rf, param_grid=parameters, n_jobs=-1, verbose=3, cv=3, scoring='f1')
CV_rf.fit(tf_idf_train, target_train)
CV_rf_best_par = CV_rf.best_params_
CV_rf_best_score = CV_rf.best_score_
print(CV_rf_best_par, 'F1 RandomForestRegressor', CV_rf_best_score)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
{'max_depth': 16, 'n_estimators': 150} F1 RandomForestRegressor 0.4134785202371685
CPU times: user 7.12 s, sys: 645 ms, total: 7.77 s
Wall time: 1min 8s


Проверим качество модели на тестовой выборке:

In [29]:
%%time

rf = RandomForestClassifier(class_weight='balanced', random_state=12345, max_depth=16, n_estimators=100)
rf.fit(tf_idf_train, target_train)

CPU times: user 4.22 s, sys: 33.6 ms, total: 4.25 s
Wall time: 4.27 s


RandomForestClassifier(class_weight='balanced', max_depth=16,
                       random_state=12345)

In [30]:
%%time

rf_predictions = rf.predict(tf_idf_test)
print('F1 RandomForest на тестовой выборке:', f1_score(target_test, rf_predictions))

F1 RandomForest на тестовой выборке: 0.40374470614458724
CPU times: user 285 ms, sys: 9.44 ms, total: 294 ms
Wall time: 310 ms


На тестовой выборке модель RandomForest показала метрику качества F1 = 0.4.

## Выводы

На первом этапе работы была произведена загрузка данных и их подготовка для дальнейшего обучения моделей. Тексты были очищены, лемматизированы и векторизированы. 

Далее были обучены 2 модели: логистическая регрессия и случайный лес. Наилучшую метрику качества F1 показала модель логистической регрессии (F1=0.73). Цель работы достигнута.